In [1]:
from ipyrad.assemble.clustmap import *
from ipyrad.assemble.cluster_across import *
import ipyparallel as ipp

In [2]:
data = ip.load_json("4-refpairtest.json")
samples = list(data.samples.values())
sample = samples[0]
force = True
ipyclient = ipp.Client()

loading Assembly: 4-refpairtest
from saved path: ~/Documents/ipyrad/tests/4-refpairtest.json


/home/deren/miniconda3/lib/python3.6/site-packages/ipyparallel/client/client.py:458: RuntimeWarning: 
            Controller appears to be listening on localhost, but not on this machine.
            If this is true, you should specify Client(...,sshserver='you@oud')
            or instruct your controller to listen on an external IP.
  RuntimeWarning)


In [3]:
data.samples

{'3J_0': <ipyrad.core.sample.Sample at 0x7f5b79e75da0>,
 '1D_0': <ipyrad.core.sample.Sample at 0x7f5b79c64f60>,
 '2H_0': <ipyrad.core.sample.Sample at 0x7f5b79c53a90>,
 '1B_0': <ipyrad.core.sample.Sample at 0x7f5b79c89630>,
 '2F_0': <ipyrad.core.sample.Sample at 0x7f5b79c89b38>,
 '3L_0': <ipyrad.core.sample.Sample at 0x7f5b79c89fd0>,
 '2E_0': <ipyrad.core.sample.Sample at 0x7f5b79c53c88>,
 '3I_0': <ipyrad.core.sample.Sample at 0x7f5b79c89128>,
 '3K_0': <ipyrad.core.sample.Sample at 0x7f5b79c175c0>,
 '2G_0': <ipyrad.core.sample.Sample at 0x7f5b79c17f28>,
 '1A_0': <ipyrad.core.sample.Sample at 0x7f5b79c17eb8>,
 '1C_0': <ipyrad.core.sample.Sample at 0x7f5b79c22978>}

In [4]:
popmins = {'1': 3, '2': 3, '3': 3}
popdict = {
    '1': ['1A_0', '1B_0', '1C_0', '1D_0'], 
    '2': ['2E_0', '2F_0', '2G_0', '2H_0'],
    '3': ['3I_0', '3J_0', '3K_0', '3L_0'],
}

data._link_populations(popdict, popmins)
data.populations

{'1': (3, ['1A_0', '1B_0', '1C_0', '1D_0']),
 '2': (3, ['2E_0', '2F_0', '2G_0', '2H_0']),
 '3': (3, ['3I_0', '3J_0', '3K_0', '3L_0'])}

In [5]:
s = Step6(data, samples, ipyclient, 123, True)
s.samples

In [6]:
s.remote_build_concats_tier1()

[####################] 100% 0:00:02 | concatenating inputs | s6 |


In [7]:
s.remote_cluster1()

[####################] 100% 0:00:03 | clustering across 1  | s6 |


In [8]:
s.remote_build_concats_tier2()


[####################] 100% 0:00:00 | concatenating inputs | s6 |


In [9]:
s.remote_cluster2()

[####################] 100% 0:00:00 | clustering 2 | s6 |


In [10]:
s.remote_build_denovo_clusters()

[#############       ]  66% 0:00:00 | building clusters    | s6 |


In [24]:
chunk = "4-refpairtest_across/4-refpairtest-tmpalign/4-refpairtest.chunk_105"
samples = s.samples
data = s.data


# data are already chunked, read in the whole thing
with open(chunk, 'rt') as infile:
    clusts = infile.read().split("//\n//\n")[:-1]    
    


In [35]:
# ensure sample name order
samples.sort(key=lambda x: x.name)
snames = [sample.name for sample in samples]

# temp arrays for filtered clusters
maxlen = 50
filter_indels = np.zeros(len(clusts), dtype=np.bool_)
filter_duples = np.zeros(len(clusts), dtype=np.bool_)

In [29]:
# create a persistent shell for running muscle in. 
proc = sps.Popen(
    ["bash"], 
    stdin=sps.PIPE, 
    stdout=sps.PIPE,
    bufsize=0, 
    #universal_newlines=True,
    )

In [36]:
## iterate over clusters until finished
allstack = []
for ldx in range(len(clusts)):
    # new alignment string for read1s and read2s
    aligned = []
    istack = []
    lines = clusts[ldx].strip().split("\n")
    names = lines[::2]
    seqs = lines[1::2]        
    align1 = []
    align2 = []

    # find duplicates and ...
    if len(names) != len(set([x.rsplit("_", 1)[0] for x in names])):
        filter_duples[ldx] = True
        istack = [consensus(seqs)]
        
    else:
        names = [">{};*{}".format(j[1:], i) for i, j in enumerate(names)]
        
        try:
            # try to split names on nnnn splitter
            clust1, clust2 = zip(*[i.split("nnnn") for i in seqs])

            # make back into strings
            cl1 = "\n".join(chain(*zip(names, clust1)))
            cl2 = "\n".join(chain(*zip(names, clust2)))

            # store allele (lowercase) info
            shape = (len(seqs), max([len(i) for i in seqs]))
            arrseqs = np.zeros(shape, dtype="S1")
            for row in range(arrseqs.shape[0]):
                seqsrow = seqs[row]
                arrseqs[row, :len(seqsrow)] = list(seqsrow)
            amask = np.char.islower(arrseqs)
            save_alleles = np.any(amask)
            
            print(save_alleles)
        
        finally:
            pass

In [32]:
def consensus(seqs):
    return seqs[0]

In [37]:
istack

['TGCAGCTTCTGAAGCCTTCAATCCTCACGTCAACATGATGCCTACATGAATCATATACTGTTTATATTATCCTTATACACAAAAGAGCCTANNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNTGTCCCTCACCTTCAGAGTACATGAGAGGTTCGACGCGTCGAATCATCTATMACAATGCGCGGCAGTTACGCCCCTCGGTATGAAAGCAAGACCCCCCCG']

In [22]:
    seeds = [
        os.path.join(
            data.dirs.across, 
            "{}-{}.htemp".format(data.name, jobid)) for jobid in jobids
        ]
    allseeds = os.path.join(data.dirs.across, "{}-x.fa".format(data.name))

    cmd1 = ['cat'] + seeds
    cmd2 = [ipyrad.bins.vsearch, '--sortbylength', '-', '--output', allseeds]
    proc1 = sps.Popen(cmd1, stdout=sps.PIPE, close_fds=True)
    proc2 = sps.Popen(cmd2, stdin=proc1.stdout, stdout=sps.PIPE, close_fds=True)
    out = proc2.communicate()
    proc1.stdout.close()

In [23]:
out

(b'', None)

In [12]:
data = s.data
jobid = 2
nthreads = 4

In [14]:
    # get files for this jobid
    catshuf = os.path.join(
        data.dirs.across, 
        "{}-{}-catshuf.fa".format(data.name, jobid))
    uhaplos = os.path.join(
        data.dirs.across, 
        "{}-{}.utemp".format(data.name, jobid))
    hhaplos = os.path.join(
        data.dirs.across, 
        "{}-{}.htemp".format(data.name, jobid))
    #logfile = os.path.join(data.dirs.across, "s6_cluster_stats.txt")

    ## parameters that vary by datatype
    ## (too low of cov values yield too many poor alignments)
    strand = "plus"
    cov = 0.75         # 0.90
    if data.paramsdict["datatype"] in ["gbs", "2brad"]:
        strand = "both"
        cov = 0.60
    elif data.paramsdict["datatype"] == "pairgbs":
        strand = "both"
        cov = 0.75     # 0.90

    ## nthreads is calculated in 'call_cluster()'
    cmd = [ipyrad.bins.vsearch,
           "-cluster_smallmem", catshuf,
           "-strand", strand,
           "-query_cov", str(cov),
           "-minsl", str(0.5),
           "-id", str(data.paramsdict["clust_threshold"]),
           "-userout", uhaplos,
           "-notmatched", hhaplos,
           "-userfields", "query+target+qstrand",
           "-maxaccepts", "1",
           "-maxrejects", "0",
           "-fasta_width", "0",
           "-threads", str(nthreads),  # "0",
           "-fulldp",
           "-usersort",
           ]
    proc = sps.Popen(cmd, stderr=sps.STDOUT, stdout=sps.PIPE)
    out = proc.communicate()
    if proc.returncode:
        raise IPyradError(out)

In [20]:
proc = sps.Popen(cmd, stderr=sps.STDOUT, stdout=sps.PIPE)
out = proc.communicate()
if proc.returncode:
    raise IPyradError(out)


In [22]:
build_concat_files(s.data, 1, [s.data.samples[i] for i in s.cgroups[1]], 123)

In [23]:
data = s.data
jobid = 1
samples = [s.data.samples[i] for i in s.cgroups[1]]
randomseed = 123

In [24]:
conshandles = [
    sample.files.consens[0] for sample in samples if 
    sample.stats.reads_consens]
conshandles.sort()
assert conshandles, "no consensus files found"


In [26]:
    ## concatenate all of the gzipped consens files
    cmd = ['cat'] + conshandles
    groupcons = os.path.join(
        data.dirs.across, 
        "{}-{}-catcons.gz".format(data.name, jobid))
    LOGGER.debug(" ".join(cmd))
    with open(groupcons, 'w') as output:
        call = sps.Popen(cmd, stdout=output, close_fds=True)
        call.communicate()

    ## a string of sed substitutions for temporarily replacing hetero sites
    ## skips lines with '>', so it doesn't affect taxon names
    subs = ["/>/!s/W/A/g", "/>/!s/w/A/g", "/>/!s/R/A/g", "/>/!s/r/A/g",
            "/>/!s/M/A/g", "/>/!s/m/A/g", "/>/!s/K/T/g", "/>/!s/k/T/g",
            "/>/!s/S/C/g", "/>/!s/s/C/g", "/>/!s/Y/C/g", "/>/!s/y/C/g"]
    subs = ";".join(subs)

    ## impute pseudo-haplo information to avoid mismatch at hetero sites
    ## the read data with hetero sites is put back into clustered data later.
    ## pipe passed data from gunzip to sed.
    cmd1 = ["gunzip", "-c", groupcons]
    cmd2 = ["sed", subs]
    LOGGER.debug(" ".join(cmd1))
    LOGGER.debug(" ".join(cmd2))

    proc1 = sps.Popen(cmd1, stdout=sps.PIPE, close_fds=True)
    allhaps = groupcons.replace("-catcons.gz", "-cathaps.fa")
    with open(allhaps, 'w') as output:
        proc2 = sps.Popen(cmd2, stdin=proc1.stdout, stdout=output, close_fds=True)
        proc2.communicate()
    proc1.stdout.close()

    ## now sort the file using vsearch
    allsort = groupcons.replace("-catcons.gz", "-catsort.fa")
    cmd1 = [ipyrad.bins.vsearch,
            "--sortbylength", allhaps,
            "--fasta_width", "0",
            "--output", allsort]
    LOGGER.debug(" ".join(cmd1))
    proc1 = sps.Popen(cmd1, close_fds=True)
    proc1.communicate()

(None, None)

In [28]:
data.dirs.across

''

In [8]:
# calculate the theading of cluster1 jobs:
ncpus = len(self.ipyclient.ids)
njobs = len(self.cgroups)
nnodes = len(self.hostd)
minthreads = 2
maxthreads = 10

In [9]:
# product
targets = [0, 1]
self.thview = self.ipyclient.load_balanced_view(targets=targets)

In [10]:
s.prepare_concats()

[####################] 100% 0:00:02 | concatenating inputs | s6 |


In [31]:
cluster1(data, 2, 4)

IPyradError: (b'vsearch v2.8.0_linux_x86_64, 15.5GB RAM, 4 cores\nhttps://github.com/torognes/vsearch\n\n\n\nUnable to open file for reading (/home/deren/Documents/ipyrad/tests/4-refpairtest_across/4-refpairtest-2-catshuf.fa)\n', None)

In [30]:


def cluster1(data, jobid, nthreads):

    # get files for this jobid
    catshuf = os.path.join(
        data.dirs.across, 
        "{}-{}-catshuf.fa".format(data.name, jobid))
    uhaplos = os.path.join(
        data.dirs.across, 
        "{}-{}.utemp".format(data.name, jobid))
    hhaplos = os.path.join(
        data.dirs.across, 
        "{}-{}.htemp".format(data.name, jobid))

    ## parameters that vary by datatype
    ## (too low of cov values yield too many poor alignments)
    strand = "plus"
    cov = 0.75         # 0.90
    if data.paramsdict["datatype"] in ["gbs", "2brad"]:
        strand = "both"
        cov = 0.60
    elif data.paramsdict["datatype"] == "pairgbs":
        strand = "both"
        cov = 0.75     # 0.90

    ## nthreads is calculated in 'call_cluster()'
    cmd = [ipyrad.bins.vsearch,
           "-cluster_smallmem", catshuf,
           "-strand", strand,
           "-query_cov", str(cov),
           "-minsl", str(0.5),
           "-id", str(data.paramsdict["clust_threshold"]),
           "-userout", uhaplos,
           "-notmatched", hhaplos,
           "-userfields", "query+target+qstrand",
           "-maxaccepts", "1",
           "-maxrejects", "0",
           "-fasta_width", "0",
           "-threads", str(nthreads),  # "0",
           "-fulldp",
           "-usersort",
           ]
    proc = sps.Popen(cmd, stderr=sps.STDOUT, stdout=sps.PIPE)
    out = proc.communicate()
    if proc.returncode:
        raise IPyradError(out)

In [12]:
if len(self.samples) < 50:
    nclusters = 1
elif len(self.samples) < 100:
    nclusters = 2
elif len(self.samples) < 200:
    nclusters = 4
elif len(self.samples) > 500:
    nclusters = 10
else:
    nclusters = int(len(self.samples) / 10)
    
    

In [10]:
self.cgroups

{0: ['1A_0', '1B_0', '1C_0', '1D_0'],
 1: ['2E_0', '2F_0', '2G_0', '2H_0'],
 2: ['3I_0', '3J_0', '3K_0', '3L_0']}

In [48]:
hosts


{'oud': [0, 1, 2, 3]}